In [1]:
import pandas as pd

df1 = pd.read_csv("data/store_sale_2019.csv", encoding='cp949')
df2 = pd.read_csv("data/store_sale_2020.csv", encoding='cp949')
df3 = pd.read_csv("data/store_sale_2021.csv", encoding='cp949')
df4 = pd.read_csv("data/store_sale_2022.csv", encoding='cp949')
df5 = pd.read_csv("data/store_sale_2023.csv", encoding='cp949')
df6 = pd.read_csv("data/store_sale_2024.csv", encoding='cp949')

# 데이터프레임 취합
df_sale = pd.concat([df1, df2, df3, df4, df5, df6])

In [2]:
# 사용할 원본 컬럼
use_cols = [
    '기준_년분기_코드',
    '상권_코드',
    '상권_코드_명',
    '당월_매출_금액',
    '당월_매출_건수',
    '주중_매출_금액',
    '주말_매출_금액',
    '연령대_20_매출_금액',
    '연령대_30_매출_금액',
    '연령대_40_매출_금액',
    '연령대_50_매출_금액',
    '연령대_60_이상_매출_금액'
]

In [3]:
# 서비스업종 코드를 날리고 상권단위로 GROUP BY SUM 작업 / 불필요한 상권구분 컬럼제거.
df_sale = df_sale.drop(columns=['상권_구분_코드', '상권_구분_코드_명', '서비스_업종_코드', '서비스_업종_코드_명'])

df_sale = df_sale.groupby(
    ['기준_년분기_코드',  '상권_코드', '상권_코드_명'],
    as_index=False
).sum(numeric_only=True)

In [4]:
# 주중 매출 비중
df_sale['주중비중'] = df_sale['주중_매출_금액'] / (df_sale['주중_매출_금액'] + df_sale['주말_매출_금액'])

# 2030세대 매출 비중
df_sale['2030_매출비중'] = (df_sale['연령대_20_매출_금액'] + df_sale['연령대_30_매출_금액']) / df_sale['당월_매출_금액']

# 매출 변화율 (전분기 대비)
df_sale['매출_변화율'] = df_sale.groupby('상권_코드')['당월_매출_금액'].pct_change()

In [5]:
# 불필요한 무한대/결측치 처리
df_sale = df_sale.replace([float('inf'), float('-inf')], None)
df_sale = df_sale.dropna(subset=['매출_변화율', '주중비중', '2030_매출비중'])

In [6]:
# 2019년도 데이터 제거
df_sale = df_sale[~df_sale['기준_년분기_코드'].astype(str).str.contains('2019')]

In [7]:
# 인덱스 리셋
df_sale = df_sale.reset_index(drop=True)

In [8]:
df_sale = df_sale[[ '기준_년분기_코드',
                    '상권_코드',
                    '상권_코드_명',
                    '당월_매출_금액',
                    '당월_매출_건수',
                    '주중_매출_금액',
                    '주말_매출_금액',
                    '연령대_20_매출_금액',
                    '연령대_30_매출_금액',
                    '연령대_40_매출_금액',
                    '연령대_50_매출_금액',
                    '연령대_60_이상_매출_금액',
                    '주중비중',
                    '매출_변화율',
                    '2030_매출비중']]

In [ ]:
# 저장 직전에 컬럼명 영문 변환
df_sale = df_sale.rename(columns={
    '기준_년분기_코드': 'year_quarter',
    '상권_코드': 'region_code',
    '상권_코드_명': 'region_name',
    '당월_매출_금액': 'monthly_sales_amount',
    '당월_매출_건수': 'monthly_sales_count',
    '주중_매출_금액': 'weekday_sales_amount',
    '주말_매출_금액': 'weekend_sales_amount',
    '연령대_20_매출_금액': 'age20_sales_amount',
    '연령대_30_매출_금액': 'age30_sales_amount',
    '연령대_40_매출_금액': 'age40_sales_amount',
    '연령대_50_매출_금액': 'age50_sales_amount',
    '연령대_60_이상_매출_금액': 'age60_sales_amount',
    '주중비중': 'weekday_ratio',
    '매출_변화율': 'sales_growth_rate',
    '2030_매출비중': 'sales_ratio_2030'
})

In [10]:
# 문자열로 변환 후 포맷 변경
df_sale['year_quarter'] = df_sale['year_quarter'].astype(str).apply(lambda x: x[:4] + 'Q' + x[4])

In [11]:
output_path = r'data/02_store_sale.csv'
df_sale.to_csv(output_path, index=False)